### Import/Load Data

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.learn.python.learn.datasets import base


# Load dataset
training_data = "iris_training.csv"
testing_data = "iris_test.csv"
training_set = base.load_csv_with_header(filename=training_data,
                                         features_dtype=np.float32, 
                                         target_dtype=np.int)
test_set = base.load_csv_with_header(filename=testing_data,
                                     features_dtype=np.float32,
                                     target_dtype=np.int)

In [ ]:
feature_name = "iris_features"
feature_columns = [tf.feature_column.numeric_column(feature_name, shape=[4])]

### Input Function

In [ ]:
def input_fn(data):
    def _fn():
        features = {feature_name: tf.constant(data.data)}
        label = tf.constant(data.target)
        return features, label
    return _fn

### Training w/ Linear Estimator

In [ ]:
classifier = tf.estimator.LinearClassifier(
    feature_columns=feature_columns,
    n_classes=3,
    model_dir="tmp/iris")

classifier.train(input_fn=input_fn(training_set),
               steps=100)

accuracy_score = classifier.evaluate(input_fn=input_fn(test_set), 
                                     steps=50)["accuracy"]
print('\n\n Accuracy: {0:f}'.format(accuracy_score))

### Training w/ DNN Estimator

In [ ]:
nn_classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[64, 12, 4],
    activation_fn=tf.nn.relu,
    dropout=0.4,
    n_classes=4,
    optimizer="Adam",    
    model_dir="tmp/irisnn")

nn_classifier.train(input_fn=input_fn(training_set),
               steps=1000)

accuracy_score = nn_classifier.evaluate(input_fn=input_fn(test_set), 
                                     steps=1000)["accuracy"]
print('\n\n Accuracy: {0:f}'.format(accuracy_score))

### Serving function

In [ ]:
feature_spec = {'iris_features': 
                tf.FixedLenFeature(shape=[4], dtype=np.float32)}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

### Export Model for Prediction

In [ ]:
feature_spec = {'iris_features': tf.FixedLenFeature(shape=[4], dtype=np.float32)}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

classifier.export_savedmodel(export_dir_base='tmp/irisnn/export', 
                            serving_input_receiver_fn=serving_fn)